In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

C:\Users\sadan\AppData\Local\Continuum\anaconda3\envs\old_env\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.
C:\Users\sadan\AppData\Local\Continuum\anaconda3\envs\old_env\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sadan\AppData\Local\Continuum\anaconda3\envs\old_env\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 

In [45]:
data = pd.read_csv("fake_real3.csv",  encoding='utf-8')

In [46]:
# Set index 
data = data.set_index("Unnamed: 0")

# Print first lines of `df` 
data.head()

,text,Label
Unnamed: 0,,
1365,أجنبي يتجنب نقد الإسلام لأن ذلك الكافر يود إظه...,fake
1499,إسرائيل بين مطرقة الجامعة العربية وسندان البرل...,fake
1621,ايران الافراج عن ثامن موظف من موظفي ال...,Real
735,أحمد كريمة عرض التماثيل في المتاحف حرام ترض...,fake
2369,تقرير باكستاني الجماعات الاسلامية توسع...,Real


In [47]:
trainDF = pd.DataFrame()
trainDF['text'] = data['text']
trainDF['label'] = data['Label']

trainDF.head()

,text,label
Unnamed: 0,,
1365,أجنبي يتجنب نقد الإسلام لأن ذلك الكافر يود إظه...,fake
1499,إسرائيل بين مطرقة الجامعة العربية وسندان البرل...,fake
1621,ايران الافراج عن ثامن موظف من موظفي ال...,Real
735,أحمد كريمة عرض التماثيل في المتاحف حرام ترض...,fake
2369,تقرير باكستاني الجماعات الاسلامية توسع...,Real


In [5]:
import re
def cleaning_hudoodtext2 (txt):
    
   
       
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    txt = re.sub(p_tashkeel,"", txt)#remove tashkeel (diactritics)
    
    txt = re.sub(r'\d+', ' ', txt) #substituting digits with 'number'
    txt = re.sub(r'[a-zA-Z]+', ' ', txt, flags=re.I) #deleting english letters
    txt = re.sub(r'\W', ' ', txt, flags = re.I) # remove sepcial characters
    txt = re.sub(r'\s+', ' ', txt) #Substituting multiple spaces with single space
    txt = re.sub(r'(?:^| )\w(?:$| )', ' ', txt) #Substituting single characters
    txt = re.sub(r'(\s+[a-zA-Z]\s+)', ' ', txt) #Substituting single characters
    



    return txt


In [6]:
#trainDF['text'] = trainDF['text'].apply(cleaning_hudoodtext2)

In [48]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size = .35)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [49]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [50]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=1500)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=1500)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=1500)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [8]:
#number of extra text based features 

trainDF['char_count'] = trainDF['text'].apply(len)
trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
trainDF['punctuation_count'] = trainDF['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
trainDF['title_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
trainDF['upper_case_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [41]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    acc = metrics.accuracy_score(valid_y,predictions)    
    confusion = confusion_matrix(valid_y,predictions)
    
    return print( classification_report(predictions, valid_y, target_names=['fake', 'Real'],digits=4),acc,confusion)



In [51]:
# Naive Bayes on Count Vectors
from sklearn.metrics import classification_report
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print( "NB, CharLevel Vectors: ", accuracy)

             precision    recall  f1-score   support

       fake     0.9936    0.9340    0.9629      1167
       Real     0.9315    0.9934    0.9614      1054

avg / total     0.9641    0.9622    0.9622      2221
 0.9621791985592075 [[1090    7]
 [  77 1047]]
NB, Count Vectors:  None
             precision    recall  f1-score   support

       fake     0.9644    0.9421    0.9532      1123
       Real     0.9422    0.9645    0.9532      1098

avg / total     0.9534    0.9532    0.9532      2221
 0.9531742458352094 [[1058   39]
 [  65 1059]]
NB, WordLevel TF-IDF:  None
             precision    recall  f1-score   support

       fake     0.9644    0.8164    0.8842      1296
       Real     0.7883    0.9578    0.8648       925

avg / total     0.8911    0.8753    0.8762      2221
 0.875281404772625 [[1058   39]
 [ 238  886]]
NB, N-Gram Vectors:  None
             precision    recall  f1-score   support

       fake     0.9435    0.9151    0.9291      1131
       Real     0.9146    0.9431

In [52]:
#segmented 
data2 = pd.read_csv("fake_real_seg.csv",  encoding='utf-8')

In [53]:
# Set index 
data2 = data2.set_index("Unnamed: 0")

# Print first lines of `df` 
data2.head()

,text,Label
Unnamed: 0,,
1187,مقتل شرط ة ايراني ين ب مواجه ة مع مهربي مخدر...,Real
2874,بعد مطالب عديد ة ب ال غائ ها بيع دائر ة ال خصخ...,fake
2699,جزائر ي و صيني ي شتبك ون ب ال سلاح ال أبيض ف...,Real
730,ال زعبي أنا نائب ل يوم ال دين ال مصدر,fake
2410,مجموع ة ال ثماني ت دعو إلى ال حزم ب شان ملف ...,Real


In [54]:
trainDF2 = pd.DataFrame()
trainDF2['text'] = data2['text']
trainDF2['label'] = data2['Label']

trainDF.head()

,text,label
Unnamed: 0,,
1365,أجنبي يتجنب نقد الإسلام لأن ذلك الكافر يود إظه...,fake
1499,إسرائيل بين مطرقة الجامعة العربية وسندان البرل...,fake
1621,ايران الافراج عن ثامن موظف من موظفي ال...,Real
735,أحمد كريمة عرض التماثيل في المتاحف حرام ترض...,fake
2369,تقرير باكستاني الجماعات الاسلامية توسع...,Real


In [55]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF2['text'], trainDF2['label'], test_size = .35)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [56]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF2['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [58]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=1500)
tfidf_vect.fit(trainDF2['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=1500)
tfidf_vect_ngram.fit(trainDF2['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=1500)
tfidf_vect_ngram_chars.fit(trainDF2['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [35]:
#number of extra text based features 

trainDF['char_count'] = trainDF['text'].apply(len)
trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
trainDF['punctuation_count'] = trainDF['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
trainDF['title_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
trainDF['upper_case_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [36]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    acc = metrics.accuracy_score(valid_y,predictions)    
    confusion = confusion_matrix(valid_y,predictions)
    
    return print( classification_report(predictions, valid_y, target_names=['fake', 'Real'],digits=4),acc,confusion)



In [59]:
# Naive Bayes on Count Vectors
from sklearn.metrics import classification_report
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print( "NB, CharLevel Vectors: ", accuracy)

             precision    recall  f1-score   support

       fake     0.9845    0.9532    0.9686      1132
       Real     0.9526    0.9843    0.9682      1083

avg / total     0.9689    0.9684    0.9684      2215
 0.9683972911963883 [[1079   17]
 [  53 1066]]
NB, Count Vectors:  None
             precision    recall  f1-score   support

       fake     0.9626    0.9496    0.9560      1111
       Real     0.9500    0.9629    0.9564      1104

avg / total     0.9563    0.9562    0.9562      2215
 0.9562076749435666 [[1055   41]
 [  56 1063]]
NB, WordLevel TF-IDF:  None
             precision    recall  f1-score   support

       fake     0.9644    0.9346    0.9493      1131
       Real     0.9339    0.9640    0.9487      1084

avg / total     0.9495    0.9490    0.9490      2215
 0.9489841986455982 [[1057   39]
 [  74 1045]]
NB, N-Gram Vectors:  None
             precision    recall  f1-score   support

       fake     0.9453    0.9225    0.9338      1123
       Real     0.9223    0.945

In [28]:
#segmented
data2 = pd.read_csv("fake_real_seg.csv",  encoding='utf-8')

In [17]:
# Naive Bayes on Count Vectors
from sklearn.metrics import classification_report
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

             precision    recall  f1-score   support

       fake       0.99      0.94      0.97       831
       Real       0.94      0.99      0.97       755

avg / total       0.97      0.97      0.97      1586

NB, Count Vectors:  None


In [12]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print ("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("LR, CharLevel Vectors: ", accuracy)

LR, Count Vectors:  0.9943253467843631
LR, WordLevel TF-IDF:  0.989281210592686
LR, N-Gram Vectors:  0.9842370744010088
LR, CharLevel Vectors:  0.9798234552332913


In [13]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)

SVM, N-Gram Vectors:  0.4943253467843632


In [14]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print ("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("RF, WordLevel TF-IDF: ", accuracy)

RF, Count Vectors:  0.9691046658259773
RF, WordLevel TF-IDF:  0.9880201765447667
